# Expriments

- Journal Version

- 2024-10-30

In [1]:
setwd('C:/CodeFields/SGD_Project/SGD_R_package/Expriments')

In [2]:
getwd()

[1] "C:/CodeFields/SGD_Project/SGD_R_package/Expriments"

In [1]:
source('../R/SGD.R')
source('../R/graph_builder.R')
source('../R/graph_analyzer.R')
source('../R/match.R')
source('../R/MMD.R')
source('../R/utils.R')

In [2]:
library(igraph)
library(doParallel)
library(foreach)
library(igraph)
library(RANN)
library(reticulate)


Attaching package: 'igraph'


The following objects are masked from 'package:stats':

    decompose, spectrum


The following object is masked from 'package:base':

    union


Loading required package: foreach

Loading required package: iterators

Loading required package: parallel



## Simulate

### 边缘中心

In [5]:
load("../Data/data/Simu/1_center_edge/SIMU_Case_CenterEdge.RData")

In [6]:
seurat_object
metadata <- seurat_object@meta.data
coordinates <- metadata[, c("spatial_x", "spatial_y")]
truth_labels <- metadata$truth_label
pred1_labels <- metadata$edge_error

Loading required package: SeuratObject

Loading required package: sp


Attaching package: 'SeuratObject'


The following objects are masked from 'package:base':

    intersect, t




An object of class Seurat 
100 features across 112 samples within 1 assay 
Active assay: RNA (100 features, 0 variable features)
 1 layer present: counts

In [7]:
params1 <-list(
  apply_gene_similarity = FALSE,
  apply_anomaly_severity_weight = FALSE,
  AD_weight = 0.7,
  sigma = 1e-2
)

In [8]:
SGD_test = SGD(
  true_labels  = truth_labels, 
  cluster_labels = pred1_labels , 
  spatial_coordinates = coordinates,
  params = params1)

构建图时间: 0.01 秒
unique_groups: A B 
unique_groups: A B 
samples_truth shape: 375 2 
samples_pred shape: 375 2 
samples_set_truth shape: 4 375 2 
               [,1]        [,2]
  [1,] 1.0000000000 0.072756729
  [2,] 0.0000000000 0.000000000
  [3,] 0.8705584902 0.157020428
  [4,] 0.9324233974 0.000000000
  [5,] 0.9124014533 0.000000000
  [6,] 0.0000000000 0.000000000
  [7,] 1.0000000000 0.050995190
  [8,] 0.9531568270 0.149872117
  [9,] 1.0000000000 0.000000000
 [10,] 0.0276796297 1.000000000
 [11,] 0.0000000000 0.147787514
 [12,] 0.9597605418 0.000000000
 [13,] 1.0000000000 0.054867340
 [14,] 1.0000000000 0.012765677
 [15,] 0.0000000000 0.000000000
 [16,] 0.7520210842 0.000000000
 [17,] 0.0000000000 0.000000000
 [18,] 0.9331459587 0.115899747
 [19,] 0.9843995102 0.012794665
 [20,] 0.0000000000 0.126246842
 [21,] 1.0000000000 0.000000000
 [22,] 1.0000000000 0.000000000
 [23,] 1.0000000000 0.031792086
 [24,] 0.0474596483 0.980186332
 [25,] 1.0000000000 0.027309200
 [26,] 1.0000000000 0.000

### 测试FP FN

In [9]:
dict_severity_levels1 <- list(
  list(name = "A", severity_level = 1),
  list(name = "B", severity_level = 2)
)

In [14]:
params2 <-list(
  apply_gene_similarity = FALSE,
  apply_anomaly_severity_weight = TRUE,
  AD_weight = 0.7,
  sigma = 1e-2,
  severity_weight_dict = dict_severity_levels1
)

In [15]:
SGD_test2 = SGD(
  true_labels  = truth_labels, 
  cluster_labels = pred1_labels , 
  spatial_coordinates = coordinates,
  params = params2)

Applying anomaly severity weight when building graphs.



Graphs built successfully.
构建图时间: 0.17 秒
unique_groups: A B 
unique_groups: A B 
num_samples: 375 
samples_truth shape: 375 2 
拟合KDE时间: 0.64 秒
拟合KDE时间: 0.44 秒
samples_set_truth shape: 1 375 2 
samples_set_pred shape: 1 375 2 
Graphs analyzed successfully.
分析图时间: 1.56 秒
计算MMD时间: 0.2 秒
SGD: 0.06567551 


### 测试Gene Similarity

In [3]:
library(anndata)
library(reticulate)
use_condaenv("Pytorch", required = TRUE)
anndata2 = read_h5ad("Data/data/Real/1-DLPFC-151673-all/DLPFC_151673_merge_matched.h5ad")

In [4]:
truth_labels <- anndata2$obs$Region
pred_label_CCST <- anndata2$obs$pred_CCST_matched_label
coordinates <- anndata2$obsm$spatial
GeneExpMatrix <- anndata2$X
GeneExpMatrix <- as.matrix(GeneExpMatrix)

In [5]:
params3 <-list(
  apply_gene_similarity = TRUE,
  apply_anomaly_severity_weight = FALSE,
  AD_weight = 0.7,
  sigma = 1e-2,
  gene_exp_matrix = GeneExpMatrix
)

In [6]:
SGD_test3 = SGD(
  true_labels  = truth_labels, 
  cluster_labels = pred_label_CCST , 
  spatial_coordinates = coordinates,
  params = params3)

Applying gene similarity weight when building graphs.



开始计算Gene similarity
计算组均值时间: 1.33 秒
获取边端点时间: 0 秒
遍历边并计算相似度时间: 46.64 秒
构建图时间: 48.53 秒
unique_groups: 1 2 3 4 5 6 7 
unique_groups: 1 2 3 4 5 6 7 
samples_truth shape: 11084 7 
samples_pred shape: 11084 7 
samples_set_truth shape: 4 11084 7 
samples_set_pred shape: 4 11084 7 
分析图时间: 15.41 秒
使用的sigma是:  0.01 
计算MMD时间: 26.7 秒
SGD: 1.536578 


## Real

### DLPFC

In [3]:
library(anndata)
library(reticulate)
use_condaenv("Pytorch", required = TRUE)
adata_DLPFC_151673 = read_h5ad("../Data/data/Real/1-DLPFC-151673-all/DLPFC_matched_final.h5ad")

In [4]:
adata_DLPFC_151673

AnnData object with n_obs × n_vars = 3611 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'Region', 'pred_leiden', 'pred_louvain', 'pred_CCST', 'pred_GraphST', 'pred_CCST_matched_label', 'pred_GraphST_matched_label', 'leiden_matched_label', 'louvain_matched_label', 'pred_GraphST_unrefine_match'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'leiden', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'spatial'
    obsm: 'X_pca', 'spatial'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [4]:
truth_labels <- adata_DLPFC_151673$obs$Region
coordinate_data <- adata_DLPFC_151673$obsm$spatial

# Pred labels

# Leiden
pred_label_leiden <- adata_DLPFC_151673$obs$leiden_matched_label

# Louvain
pred_label_louvain <- adata_DLPFC_151673$obs$louvain_matched_label

# CCST
pred_label_CCST <- adata_DLPFC_151673$obs$pred_CCST_matched_label

# GraphST refine
pred_label_GraphST_refine <- adata_DLPFC_151673$obs$pred_GraphST_matched_label

# GraphST unrefine
pred_label_GraphST_unrefine <- adata_DLPFC_151673$obs$pred_GraphST_unrefine_match

In [5]:
# Leiden
param_DLPFC <-list(
  apply_gene_similarity = FALSE,
  apply_anomaly_severity_weight = FALSE,
  AD_weight = 0.7,
  sigma = 0.1
)

In [6]:
SGD_leiden = SGD(
  true_labels  = truth_labels, 
  cluster_labels = pred_label_leiden , 
  spatial_coordinates = coordinate_data,
  params = param_DLPFC)

构建图时间: 0.03 秒
unique_groups: 1 2 3 4 5 6 7 
unique_groups: 1 2 3 4 5 6 7 
samples_truth shape: 11084 7 
samples_pred shape: 11084 7 
samples_set_truth shape: 4 11084 7 
samples_set_pred shape: 4 11084 7 
分析图时间: 11.96 秒
使用的sigma是:  0.1 
计算MMD时间: 10.64 秒
SGD: 1.147963 


In [7]:
SGD_louvain = SGD(
  true_labels  = truth_labels, 
  cluster_labels = pred_label_louvain , 
  spatial_coordinates = coordinate_data,
  params = param_DLPFC)

构建图时间: 0.02 秒
unique_groups: 1 2 3 4 5 6 7 
unique_groups: 1 2 3 4 5 6 7 
samples_truth shape: 11084 7 
samples_pred shape: 11084 7 
samples_set_truth shape: 4 11084 7 
samples_set_pred shape: 4 11084 7 
分析图时间: 10.22 秒
使用的sigma是:  0.1 
计算MMD时间: 10.92 秒
SGD: 1.285062 


In [8]:
SGD_CCST = SGD(
  true_labels  = truth_labels, 
  cluster_labels = pred_label_CCST , 
  spatial_coordinates = coordinate_data,
  params = param_DLPFC)

构建图时间: 0.02 秒
unique_groups: 1 2 3 4 5 6 7 
unique_groups: 1 2 3 4 5 6 7 
samples_truth shape: 11084 7 
samples_pred shape: 11084 7 
samples_set_truth shape: 4 11084 7 
samples_set_pred shape: 4 11084 7 
分析图时间: 9.72 秒
使用的sigma是:  0.1 
计算MMD时间: 10.65 秒
SGD: 0.7694948 


In [9]:
SGD_GraphST_refine = SGD(
  true_labels  = truth_labels, 
  cluster_labels = pred_label_GraphST_refine , 
  spatial_coordinates = coordinate_data,
  params = param_DLPFC)

构建图时间: 0.02 秒
unique_groups: 1 2 3 4 5 6 7 
unique_groups: 1 2 3 4 5 6 7 
samples_truth shape: 11084 7 
samples_pred shape: 11084 7 
samples_set_truth shape: 4 11084 7 
samples_set_pred shape: 4 11084 7 
分析图时间: 10.03 秒
使用的sigma是:  0.1 
计算MMD时间: 10.67 秒
SGD: 0.4716234 


In [10]:
SGD_GraphST_unrefine = SGD(
  true_labels  = truth_labels, 
  cluster_labels = pred_label_GraphST_unrefine , 
  spatial_coordinates = coordinate_data,
  params = param_DLPFC)

构建图时间: 0.02 秒
unique_groups: 1 2 3 4 5 6 7 
unique_groups: 1 2 3 4 5 6 7 
samples_truth shape: 11084 7 
samples_pred shape: 11084 7 
samples_set_truth shape: 4 11084 7 
samples_set_pred shape: 4 11084 7 
分析图时间: 10.08 秒
使用的sigma是:  0.1 
计算MMD时间: 10.89 秒
SGD: 0.846426 


### hBC

### MERFISH

In [3]:
library(anndata)
library(reticulate)
use_condaenv("Pytorch", required = TRUE)
adata_MERFISH = read_h5ad("../Data/data/Real/3-MERFISH-all/Data27_matched.h5ad")

In [4]:
# truth labels
truth_labels <- adata_MERFISH$obs$ground_truth
coordinate_data <- adata_MERFISH$obsm$spatial

# pred labels

# GraphST
pred_label_GraphST <- adata_MERFISH$obs$GraphST_matched_label

# STAGATE
pred_label_STAGATE <- adata_MERFISH$obs$STAGATE_matched_label

In [5]:
param_MERFISH <-list(
  apply_gene_similarity = FALSE,
  apply_anomaly_severity_weight = FALSE,
  AD_weight = 0.7,
  sigma = 0.1
)

SGD_GraphST = SGD(
  true_labels = truth_labels, 
  cluster_labels = pred_label_GraphST , 
  spatial_coordinates = coordinate_data,
  params = param_MERFISH)

构建图时间: 0.03 秒
unique_groups: 1 2 3 4 5 6 7 8 
unique_groups: 1 2 3 4 5 6 7 8 
samples_truth shape: 20414 8 
samples_pred shape: 20414 8 
samples_set_truth shape: 4 20414 8 
samples_set_pred shape: 4 20414 8 
分析图时间: 30.08 秒
使用的sigma是:  0.1 
计算MMD时间: 59.58 秒
SGD: 0.7220541 


In [6]:
SGD_STAGATE = SGD(
  true_labels  = truth_labels, 
  cluster_labels = pred_label_STAGATE , 
  spatial_coordinates = coordinate_data,
  params = param_MERFISH)

构建图时间: 0.04 秒
unique_groups: 1 2 3 4 5 6 7 8 
unique_groups: 1 2 3 4 5 6 7 8 
samples_truth shape: 20414 8 
samples_pred shape: 20414 8 
samples_set_truth shape: 4 20414 8 
samples_set_pred shape: 4 20414 8 
分析图时间: 26.86 秒
使用的sigma是:  0.1 
计算MMD时间: 64.06 秒
SGD: 1.065131 
